# Chapter Seven - Data Processing with Optimus 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../Optimus")

In [4]:
from optimus import Optimus

In [31]:
import re
size="4G"
size = size.upper()
size = re.sub(r'([KMGT]?B?)', r' \1', size)
print(size)
number, unit = [string.strip() for string in size.split()]
print(number)

 4 G 
4


In [39]:
engine = "dask"
op = Optimus(engine, n_workers=1, threads_per_worker=8, silence_logs=30, memory_limit="4 GB")
conn = op.connect.s3(endpoint_url="https://nyc3.digitaloceanspaces.com", bucket="bumblebee", key="H4MX3NNBKJJN65JNXUXM", secret="fhZFWr++ltSHsN/KKqKw6JevlXd2yf/OWt9Q0qqB6D8")
df = op.load.file("store.csv", conn=conn)
df = df.repartition("auto").execute()

4G


In [64]:
from optimus import Optimus
op = Optimus("dask")

In [65]:
op.client

Client Scheduler: inproc://192.168.100.4/19556/1 Dashboard: http://192.168.100.4:8787/status,Cluster Workers: 1 Cores: 8 Memory: 4.00 GB


In [25]:
import numpy as np
df = op.create.dataframe({"A":["",2,3,45,6,2,np.nan],"B":[1,2,3,45,6,2,np.nan],"C":[1,2,3,45,6,2,np.nan],"D":[1,2,3,45,6,2,np.nan],"E":[1,2,3,45,6,2,np.nan]})
df.print()

A                     B            C            D            E
(object)      (float64)    (float64)    (float64)    (float64)
----------  -----------  -----------  -----------  -----------
                      1            1            1            1
2                     2            2            2            2
3                     3            3            3            3
45                   45           45           45           45
6                     6            6            6            6
2                     2            2            2            2
nan                 nan          nan          nan          nan


C:\Users\argenisleon\Anaconda3\lib\site-packages\dask\dataframe\core.py:6181: UserWarning: Insufficient elements for `head`. 10 elements requested, only 7 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


In [62]:
display(df.cols.move(["D","E"],"after",["B"]).cols.names())
display(df.cols.move(["A","B"],"after",["D"]).cols.names())
display(df.cols.move(["E","D"],"before",["B"]).cols.names())
display(df.cols.move(["A","B"],"before",["D"]).cols.names())
display(df.cols.move(["A"],"end").cols.names())
display(df.cols.move(["D"],"beginning").cols.names())

col_name E 2
col_name D 2


['A', 'B', 'D', 'E', 'C']

col_name B 4
col_name A 4


['C', 'D', 'E', 'B', 'A']

col_name D 1
col_name E 1


...

['C', 'D', 'E', 'A', 'B']

col_name A 5


['B', 'C', 'D', 'E', 'A']

col_name D 0


['D', 'A', 'B', 'C', 'E']

In [7]:
df.cols.impute("A",strategy="mean").print()

          A
  (float64)
-----------
    1
    2
    3
   45
    6
    2
    9.83333


In [8]:
df.cols.impute("A",strategy="median").print()

          A
  (float64)
-----------
        1
        2
        3
       45
        6
        2
        2.5


In [9]:
print(df.cols.impute("A",strategy="most_frequent"))

          A
  (float64)
-----------
          1
          2
          3
         45
          6
          2
          2


In [10]:
print(df.cols.impute("A",strategy="constant", fill_value=1))

          A
  (float64)
-----------
          1
          2
          3
         45
          6
          2
          1


In [108]:
df = op.create.dataframe({"A":[1,2,3,45,6,2,np.nan], "B":["Optimus","Bumblebee","Eject","Optimus","Bumblebee","Eject",np.nan]})
print(df.cols.impute("B",strategy="most_frequent"))

          A  B
  (float64)  (object)
-----------  ----------
          1  Optimus
          2  Bumblebee
          3  Eject
         45  Optimus
          6  Bumblebee
        -50  Eject
        nan  Bumblebee


## Outliers

### Tukey

In [122]:
df = op.create.dataframe({"A":[1,2,3,45,6,-50,np.nan], "B":["Optimus","Bumblebee","Eject","Optimus","Bumblebee","Eject",np.nan]})

In [123]:
df

A 1 (float64) not nullable,B 2 (object) not nullable
1.0,Optimus
2.0,Bumblebee
3.0,Eject
45.0,Optimus
6.0,Bumblebee
-50.0,Eject
nan,nan


In [109]:
df.outliers.tukey("A").info()

{'count_outliers': 2,
 'count_non_outliers': 4,
 'lower_bound': -4.75,
 'lower_bound_count': 1,
 'upper_bound': 11.25,
 'upper_bound_count': 1,
 'q1': 1.25,
 'median': 2.5,
 'q3': 5.25,
 'iqr': 4.0}

In [110]:
print(df.outliers.tukey("A").select())

          A  B
  (float64)  (object)
-----------  ----------
         45  Optimus
        -50  Eject


In [111]:
print(df.outliers.tukey("A").drop())

          A  B
  (float64)  (object)
-----------  ----------
          1  Optimus
          2  Bumblebee
          3  Eject
          6  Bumblebee
        nan  nan


In [112]:
## Not outliers count
df.outliers.tukey("A").non_outliers_count()

4

In [113]:
## Outliers count
df.outliers.tukey("A").count()

2

In [114]:
## Wiskers
df.outliers.tukey("A").whiskers()

{'lower_bound': -4.75,
 'upper_bound': 11.25,
 'q1': 1.25,
 'median': 2.5,
 'q3': 5.25,
 'iqr': 4.0}

In [115]:
print(df.outliers.tukey("A").select_lower_bound())

          A  B
  (float64)  (object)
-----------  ----------
        -50  Eject


In [116]:
print(df.outliers.tukey("A").select_upper_bound())

          A  B
  (float64)  (object)
-----------  ----------
         45  Optimus


### Zscore

In [171]:
df = op.create.dataframe({"A":[1,2,3,31,6,-21,np.nan,2,4,6]})
import numpy as np
df.print()

          A
  (float64)
-----------
          1
          2
          3
         31
          6
        -21
        nan
          2
          4


In [213]:
df

A 1 (float64) not nullable
1.0
2.0
3.0
31.0
6.0
-21.0
nan
2.0
4.0


In [253]:
threshold=2
print(df.outliers.z_score("A", threshold).select())

          A
  (float64)
-----------
         31
        -21


In [257]:
print(df.cols.z_score("A"))

          A
  (float64)
-----------
 -0.224362
 -0.143592
 -0.0628213
  2.19875
  0.17949
 -2.00131
nan
 -0.143592
  0.017949


In [258]:
print(df.outliers.z_score("A", threshold).drop())

          A
  (float64)
-----------
          1
          2
          3
          6
        nan
          2
          4
          6


In [251]:
print(df.outliers.z_score("A", threshold).select_lower_bound())

          A
  (float64)
-----------
        -21


In [252]:
print(df.outliers.z_score("A", threshold).select_upper_bound())

          A
  (float64)
-----------
         31


In [247]:
df.outliers.z_score("A", threshold).non_outliers_count()
df.outliers.z_score("A", threshold).count()
df.outliers.z_score("A", threshold).info()
# df.outliers.z_score("A", threshold).drop()

df           A
  (float64)
-----------
          1
          2
          3
         31
          6
        -21
        nan
          2
          4


A 1 (float64) not nullable
nan


### Modified Z Score

In [290]:
df = op.create.dataframe({"A":[1,2,3,31,6,-21,np.nan,2,4,6]})

In [291]:
df.cols.mad()

2.0

In [315]:
threshold=3.5
print(df.outliers.modified_z_score("A", threshold).select())

          A
  (float64)
-----------
         31
        -21


In [293]:
print(df.cols.modified_z_score("A"))

          A
  (float64)
-----------
    0.6745
    0.33725
    0
    9.443
    1.01175
    8.094
  nan
    0.33725
    0.33725


In [316]:
print(df.outliers.modified_z_score("A", threshold).drop())

          A
  (float64)
-----------
          1
          2
          3
          6
        nan
          2
          4
          6


In [326]:
print(df.outliers.modified_z_score("A", threshold).info())
print(df.outliers.modified_z_score("A", threshold).count())
print(df.outliers.modified_z_score("A", threshold).non_outliers_count())

{'count_outliers': 2, 'count_non_outliers': 7, 'max_m_z_score': 31.0}
2
7


### MAD

In [327]:
print(df.outliers.mad("A", threshold).info())

{'count_outliers': 2, 'count_non_outliers': 7, 'lower_bound': -4.0, 'lower_bound_count': 1, 'upper_bound': 10.0, 'upper_bound_count': 1}


In [329]:
threshold=1
df.outliers.mad("A", threshold).select()
# df.outliers.mad("A", threshold).non_outliers_count()
# df.outliers.mad("A", threshold).count()
# df.outliers.mad("A", threshold).info()
# df.outliers.mad("A", threshold).drop()

A 1 (float64) not nullable
31.0
6.0
-21.0
6.0


In [13]:
df

A 1 (float64) not nullable
1.0
2.0
3.0
45.0
6.0
7.0
nan


## Binning

In [5]:
import numpy as np
df = op.create.dataframe({"A":[1,2,3,31,6,-21,np.nan,2,4,6]})

In [6]:
print(df)

          A
  (float64)
-----------
          1
          2
          3
         31
          6
        -21
        nan
          2
          4


In [7]:
import pandas as pd
data = pd.DataFrame({'value':[2,45, -23, 85, 28, 2, 35, -12]})
data['log+1'] = (data['value']+1).transform(np.log)
data

,value,log+1
0,2,1.098612
1,45,3.828641
2,-23,NaN
3,85,4.454347
4,28,3.367296
5,2,1.098612
6,35,3.583519
7,-12,NaN


In [357]:
print(df.cols.cut("A",bins = [0,4,6,35] ,labels = ["low", "medium","high"]))

A
(category)
------------
low
low
low
high
medium
nan
nan
low
low


In [360]:
df = op.create.dataframe({"A":["Maracaibo","Caracas","CDMX","Monterrey","Bogota"]})
print(df.cols.cut("A",["Maracaibo","Caracas","CDMX","Monterrey","Bogota"], labels = ["Venezuela","Venezuela","Mexico","Mexico","Colombia"]))

A
(object)
----------
Venezuela
Venezuela
Mexico
Mexico
Colombia


## Variable Transformation

In [7]:
df = op.create.dataframe({"A":[1,2,3,45,6,7,np.nan]})
print(df.cols.log("A"))

          A
  (float64)
-----------
   0
   0.30103
   0.477121
   1.65321
   0.778151
   0.845098
 nan


In [8]:
print(df.cols.sqrt("A"))

          A
  (float64)
-----------
    1
    1.41421
    1.73205
    6.7082
    2.44949
    2.64575
  nan


In [9]:
print(df.cols.reciprocal("A"))

          A
  (float64)
-----------
  1
  0.5
  0.333333
  0.0222222
  0.166667
  0.142857
nan


In [11]:
print(df.cols.pow("A", 2))

          A
  (float64)
-----------
          1
          4
          9
       2025
         36
         49
        nan


In [12]:
print(df.cols.pow("A", 3))

          A
  (float64)
-----------
          1
          8
         27
      91125
        216
        343
        nan


In [31]:
df.cols.hist("A",)["hist"]

{'A': [{'lower': 1.0, 'upper': 3.3157894736842106, 'count': 3},
  {'lower': 3.3157894736842106, 'upper': 5.631578947368421, 'count': 0},
  {'lower': 5.631578947368421, 'upper': 7.947368421052632, 'count': 2},
  {'lower': 7.947368421052632, 'upper': 10.263157894736842, 'count': 0},
  {'lower': 10.263157894736842, 'upper': 12.578947368421053, 'count': 0},
  {'lower': 12.578947368421053, 'upper': 14.894736842105264, 'count': 0},
  {'lower': 14.894736842105264, 'upper': 17.210526315789473, 'count': 0},
  {'lower': 17.210526315789473, 'upper': 19.526315789473685, 'count': 0},
  {'lower': 19.526315789473685, 'upper': 21.842105263157897, 'count': 0},
  {'lower': 21.842105263157897, 'upper': 24.157894736842106, 'count': 0},
  {'lower': 24.157894736842106, 'upper': 26.473684210526315, 'count': 0},
  {'lower': 26.473684210526315, 'upper': 28.789473684210527, 'count': 0},
  {'lower': 28.789473684210527, 'upper': 31.10526315789474, 'count': 0},
  {'lower': 31.10526315789474, 'upper': 33.4210526315

In [ ]:
df = op.create.dataframe({"A":[1,2,3,45,6,7,np.nan]})
df.cols.log("A")

## One-hot encoding

In [50]:
df = op.create.dataframe({"A":["Optimus","Bumblebee","Eject", "Megatron"], "B":["Transformer","Transformer","Transformer","Decepticon"]})

In [51]:
print(df)

A           B
(object)    (object)
----------  -----------
Optimus     Transformer
Bumblebee   Transformer
Eject       Transformer
Megatron    Decepticon


In [52]:
print(df.encoding.one_hot_encoder("B"))

A           B              B_Decepticon    B_Transformer
(object)    (object)            (uint8)          (uint8)
----------  -----------  --------------  ---------------
Optimus     Transformer               0                1
Bumblebee   Transformer               0                1
Eject       Transformer               0                1
Megatron    Decepticon                1                0


In [18]:
df = op.create.dataframe({"A":[1.12,3.2,4.35,6.3,7.3,np.nan]})

In [19]:
df

A 1 (float64) not nullable
1.12
3.2
4.35
6.3
7.3
nan


## splitting

In [60]:
df = op.create.dataframe({"A":["Argenis Leon","Luis Aguirre","Favio Vasquez",np.nan]})

In [65]:
# df.cols.unnest("A"," ", index=0, drop=True)
print(df.cols.unnest("A"," ", drop=True))

A_0         A_1
(object)    (object)
----------  ----------
Argenis     Leon
Luis        Aguirre
Favio       Vasquez
nan


In [66]:
df = op.create.dataframe({"A":["10/04/1980","20/05/1995","01/08/1990",np.nan]})

In [67]:
print(df.cols.unnest("A","/", splits=3, output_cols=["day","month","year"]))

A                  day       month        year
(object)      (object)    (object)    (object)
----------  ----------  ----------  ----------
10/04/1980          10          04        1980
20/05/1995          20          05        1995
01/08/1990           1          08        1990
nan                nan


## Scaling

In [73]:
df = op.create.dataframe({"A":[1.12,3.2,4.35,6.3,7.3,np.nan]})

In [74]:
print(df.cols.standard_scaler("*"))

          A
  (float64)
-----------
 -1.51526
 -0.569926
 -0.0472666
  0.838981
  1.29347
nan


In [70]:
print(df.cols.min_max_scaler("*"))

          A
  (float64)
-----------
   0
   0.33657
   0.522654
   0.838188
   1
 nan


In [69]:
print(df.cols.max_abs_scaler("*"))

          A
  (float64)
-----------
   0.153425
   0.438356
   0.59589
   0.863014
   1
 nan


## String to index, index to string

In [84]:
df = op.create.dataframe({"A":["Optimus","Bumblebee","Eject","Optimus","Eject"]})

In [87]:
print(df)

A
(object)
----------
Optimus
Bumblebee
Eject
Optimus
Eject


In [86]:
print(df.cols.string_to_index())

A             A_string_to_index
(object)                (int32)
----------  -------------------
Optimus                       2
Bumblebee                     0
Eject                         1
Optimus                       2
Eject                         1


In [51]:
# df = op.create.dataframe({"A":["Optimus","Bumblebee","Eject",np.nan]})
df.cols.index_to_string("A")

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.